In [5]:
import pandas as pd
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Trainer, TrainingArguments
from PIL import Image
from tqdm import tqdm
import shutil
import torch
import os
import numpy as np
from torch.utils.data import Dataset

# Cargar el CSV
csv_file = r'data\output\divide_images_train\finetuning.csv'
df = pd.read_csv(csv_file)

# Eliminar filas donde el texto está vacío
df = df.dropna(subset=['text'])

# Definir el procesador y el modelo preentrenado
processor = TrOCRProcessor.from_pretrained('qantev/trocr-large-spanish')
model = VisionEncoderDecoderModel.from_pretrained('qantev/trocr-large-spanish')

# Configurar los datos de entrenamiento
train_texts = df['text'].tolist()  # Usamos 'text' como la descripción del texto en el CSV
train_images = df['image_dir'].tolist()  # Usamos 'image_dir' como la ruta a las imágenes en el CSV

# Definir el Dataset personalizado
class CustomDataset(Dataset):
    def __init__(self, image_paths, texts, processor):
        self.image_paths = image_paths
        self.texts = texts
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        text = self.texts[idx]
        
        # Abrir la imagen con PIL
        with Image.open(image_path) as image:
            # Convertir la imagen a un numpy array y luego a tensor
            image_array = np.array(image)
            # Procesar la imagen con el TrOCRProcessor
            processed_input = self.processor(images=image_array, return_tensors="pt")
            pixel_values = processed_input['pixel_values'].squeeze()  # Quitar dimensión extra

        target = self.processor(text=text, return_tensors="pt", padding="max_length", truncation=True).input_ids.squeeze()
        
        return {'pixel_values': pixel_values, 'labels': target}

# Crear el dataset personalizado
train_dataset = CustomDataset(train_images, train_texts, processor)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',  # Especificar el directorio de salida
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=500,
)

# Inicializar el Trainer para el fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=processor,
)

# Fine-tuning del modelo
trainer.train()

# Guardar el modelo finetuneado y el procesador
trainer.save_model(r'models\modelo_finetuneado_trOCR')
processor.save_pretrained(r'models\procesador_finetuneado_trOCR')


  0%|          | 0/198 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'no_repeat_ngram_size': 3}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


{'train_runtime': 3487.3333, 'train_samples_per_second': 0.113, 'train_steps_per_second': 0.057, 'train_loss': 0.30971344071205215, 'epoch': 3.0}


[]

In [ ]:

# Borrar el directorio 'microsoft/trocr-base-handwritten'
shutil.rmtree('microsoft/trocr-base-handwritten')

print("Eliminado el directorio 'microsoft/trocr-base-handwritten' después de completar el proceso.")

# Ruta donde se almacena la caché (reemplaza 'ruta_de_la_cache' con la ruta real)
#Daniel cache_path = r'C:\Users\Jack\.cache\huggingface\hub\models--microsoft--trocr-large-handwritten'
#Carlos
cache_path = r'C:\Users\carlo\.cache\huggingface\hub\models--microsoft--trocr-large-handwritten'

# Verificar si la ruta existe y es un directorio
if os.path.exists(cache_path) and os.path.isdir(cache_path):
    # Borrar todos los archivos y subdirectorios dentro de la caché
    shutil.rmtree(cache_path)
    print(f"Caché en {cache_path} borrada correctamente.")
else:
    print(f"No se encontró la caché en {cache_path}.")
